In [1]:
include("../src/Utero.jl")
using .Utero 
using MLDatasets

In [2]:
TrainData = MNIST(:train)
ValidData = MNIST(:test)

dataset MNIST:
  metadata  =>    Dict{String, Any} with 3 entries
  split     =>    :test
  features  =>    28×28×10000 Array{Float32, 3}
  targets   =>    10000-element Vector{Int64}

In [3]:
densenet = Sequential(
    Dense(28*28, 16, acf=ReLU), 
    Dense(16, 16, acf=ReLU), 
    Dense(16, 16, acf=ReLU), 
    Dense(16, 16, acf=ReLU), 
    Dense(16, 10), 
    Softmax)

Sequential(Any[Dense([0.04241809678528948 0.029776287898559135 … -0.01915552483484053 0.1089056815432534; -0.005883740108774665 -0.02224987211198783 … -0.09933171745764094 0.010266067963347683; … ; -0.008981598157908239 0.13213509396988685 … 0.007941338642681175 -0.06754119848189613; -0.07284741460554266 0.05452077252712297 … 0.004180865712543068 -0.007278756586173578], [0.0; 0.0; … ; 0.0; 0.0;;], Main.Utero.ReLU), Dense([-0.0036723845708029395 0.48140151692113514 … -0.05791823464569621 -0.3845541506187205; 0.158109995406935 0.13911527796129083 … -0.1949258739953027 -0.44280160599483576; … ; 0.48025593279576567 0.16046264391563014 … -0.046594683696194086 0.020341905780721786; 0.1495644086001556 -0.03800143343370388 … -0.2923537170267285 0.5318102046744559], [0.0; 0.0; … ; 0.0; 0.0;;], Main.Utero.ReLU), Dense([0.07531092325463928 0.09461818694856024 … -0.22834912421924816 0.09705566700584355; 0.07202073785747382 0.22161141556444627 … -0.11336888156127868 0.38048233149828414; … ; -0.2322

In [4]:
predicted = []
@show size(TrainData.features)
X = reshape(TrainData.features, 60000, 28*28)
@show size(X)
predicted = zeros(60000, 10)
for (i, x) in enumerate(eachrow(X))
    predicted[i, :] = densenet(x)
end
@show size(predicted)
Accuracy(predicted, TrainData.targets)

size(TrainData.features) = (28, 28, 60000)
size(X) = (60000, 784)


size(predicted) = (60000, 10)
pred = 7
label = 5
pred = 5
label = 0
pred = 7
label = 4
pred = 7
label = 1
pred = 7
label = 9
pred = 5
label = 2
pred = 5
label = 1
pred = 5
label = 3
pred = 5
label = 1
pred = 7
label = 4
pred = 7
label = 3
pred = 7
label = 5
pred = 7
label = 3
pred = 5
label = 6
pred = 7
label = 1
pred = 7
label = 7
pred = 7
label = 2
pred = 7
label = 8
pred = 7
label = 6
pred = 7
label = 9
pred = 5
label = 4
pred = 5
label = 0
pred = 7
label = 9
pred = 7
label = 1
pred = 7
label = 1
pred = 7
label = 2
pred = 7
label = 4
pred = 7
label = 3
pred = 5
label = 2
pred = 5
label = 7


0.1027

In [5]:
ctx = ⬅Context()
AddParams!(ctx, densenet)

# Perform gradient steps
for i in 1:5
    for (x, y) in DataIterator(TrainData.features, TrainData.targets)
        x = reshape(x, 28*28)
        y = OneHot(Int64(y+1), 10)
        (out, paramgrads) = ForwardBackward!(ctx, x -> MeanSquaredError(densenet(x), y), x)
        GradientStep!(0.001, ctx.Params, paramgrads)
    end
    print("Iter Done")
end
Untrack!(densenet)

Iter DoneIter Done

Iter DoneIter Done

Iter Done

In [6]:
predicted = []
@show size(TrainData.features)
X = reshape(TrainData.features, 28*28, 60000)
@show size(X)
probabilities = zeros(60000, 10)
for (i, x) in enumerate(eachcol(X))
    probabilities[i, :] = densenet(x)
end
Accuracy(probabilities, TrainData.targets.+1)


size(TrainData.features) = (28, 28, 60000)
size(X) = (784, 60000)


pred = 1
label = 6
pred = 1
label = 1
pred = 1
label = 5
pred = 1
label = 2
pred = 1
label = 10
pred = 1
label = 3
pred = 1
label = 2
pred = 1
label = 4
pred = 1
label = 2
pred = 1
label = 5
pred = 1
label = 4
pred = 1
label = 6
pred = 1
label = 4
pred = 1
label = 7
pred = 1
label = 2
pred = 1
label = 8
pred = 1
label = 3
pred = 1
label = 9
pred = 1
label = 7
pred = 1
label = 10
pred = 1
label = 5
pred = 1
label = 1
pred = 1
label = 10
pred = 1
label = 2
pred = 1
label = 2
pred = 1
label = 3
pred = 1
label = 5
pred = 1
label = 4
pred = 1
label = 3
pred = 1
label = 8


0.09871666666666666